In [1]:
from collections import Counter
import json
import pandas as pd


species_df = pd.read_csv('data/full_many_per_image_cleaned.csv')
non_df = pd.read_csv('data/nonspecies_many_per_image_cleaned.csv')

In [2]:
species_df

,filename,species,captions,parent_taxon,subclass_of,source
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,['Q127960'],['Q6667323'],wikimedia
1,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,['Q127960'],['Q6667323'],wikimedia
2,taxo_data_images/Squirrel_posing.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,"['Q281124', 'Q18215502']",NaN,wikimedia
3,taxo_data_images/Ekorre_8218.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,"['Q281124', 'Q18215502']",NaN,wikimedia
4,taxo_data_images/Hepatitis_B_virus_1.jpg,Hepatitis B virus,['This is an example of species Hepatitis B vi...,['Q6746239'],NaN,wikimedia
...,...,...,...,...,...,...
221198,taxo_data_images/Psalidognathus_sallei_2.jpg,Psalidognathus sallei,['This is an example of species Psalidognathus...,['Q1756213'],NaN,wikimedia
221199,taxo_data_images/Hemiptarsenus_jilinus_(10.389...,Hemiptarsenus jilinus,['This is an example of species Hemiptarsenus ...,['Q13983711'],NaN,wikimedia
221200,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,['Q157069'],NaN,wikimedia
221201,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,['Q2946582'],NaN,wikimedia


In [3]:
from ast import literal_eval
non_df['parent_taxon'] = non_df['subclass_of'].map(lambda x: literal_eval(x) if type(x) is not float else []) + non_df['parent_taxon'].map(lambda x: literal_eval(x) if type(x) is not float else [])
species_df['parent_taxon'] = species_df['subclass_of'].map(lambda x: literal_eval(x) if type(x) is not float else []) + species_df['parent_taxon'].map(lambda x: literal_eval(x) if type(x) is not float else [])

In [4]:
non_df = non_df.drop('subclass_of', axis=1)
species_df = species_df.drop('subclass_of', axis=1)

In [5]:
non_df[non_df['id']=="Q127960"]

,id,taxon,parent_taxon,caption
100835,Q127960,Panthera,[Q230177],genus of mammals


In [6]:
species_df = species_df.explode('parent_taxon')
non_df = non_df.explode('parent_taxon')

In [7]:
len(non_df.iloc[:199671]['parent_taxon'].unique())

74781

In [8]:
visited = {}
can_animalia = {}

def find_animalia(taxon, depth=1):
    if depth==1:
        print('-', end='')
    if taxon=="Q729":
        return 1
    visited[taxon] = True
    li = non_df.loc[non_df.id==taxon, 'parent_taxon'].values.tolist()
    out = -1
    o = []
    for el in li:
        if visited.get(el, False):
            out = can_animalia.get(el, -1)
        else:
            out = find_animalia(el, depth=depth+1)
        if out!=-1:
            print(taxon)
            if out+1 < can_animalia.get(taxon, 100000):
                can_animalia[taxon] = out+1
            o.append(out+1)
    return min(o) if len(o) > 0 else -1

In [9]:
len(can_animalia)

0

In [10]:
len(non_df['parent_taxon'].unique())

156970

In [11]:
non_df.iloc[:5]['parent_taxon']

0        Q7239
0       Q19088
0     Q2943473
1        Q1359
1    Q82244029
Name: parent_taxon, dtype: object

In [12]:
non_df.iloc[:5]['parent_taxon'].map(find_animalia)

----Q43806
Q43806
Q1360
Q5174
Q151542
Q5174
Q3055825
Q5174
Q5173
Q5173
Q5171
Q3059449
Q5171
Q5176
Q1360
Q134870
Q1360
Q15974773
Q1360
Q1359
Q1372085
Q112271405
Q1359
-Q60712470
Q60712432
Q60791026
Q60790933
Q82244029


0   -1
0   -1
0   -1
1    4
1    9
Name: parent_taxon, dtype: int64

In [13]:
# animalia_reach = non_df['parent_taxon'].map(lambda x: find_animalia(x))

In [14]:
wgenus = pd.merge(species_df, non_df, left_on='parent_taxon', right_on='id', how='left').rename({'taxon': 'genus', 'caption': 'genus_caption', 'parent_taxon_y': 'genus_parent'}, axis=1)

In [15]:
wgenus

,filename,species,captions,parent_taxon_x,source,id,genus,genus_parent,genus_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q6667323,wikimedia,NaN,NaN,NaN,NaN
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals
2,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q6667323,wikimedia,NaN,NaN,NaN,NaN
3,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals
4,taxo_data_images/Squirrel_posing.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,Q281124,Sciurus,Q2294121,genus of mammals
...,...,...,...,...,...,...,...,...,...
240124,taxo_data_images/Hemiptarsenus_jilinus_(10.389...,Hemiptarsenus jilinus,['This is an example of species Hemiptarsenus ...,Q13983711,wikimedia,Q13983711,Hemiptarsenus,Q675905,genus of insects
240125,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q756,genus of plants
240126,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants
240127,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects


In [16]:
wgenus = wgenus.loc[~((wgenus['filename'].duplicated(False)) & (wgenus['genus'].isna()))]

In [17]:
wnew = pd.merge(wgenus, non_df, left_on='genus_parent', right_on='id', how='left').rename({'taxon': 'taxon_b', 'caption': 'taxon_b_caption', 'parent_taxon': 'taxon_b_parent'}, axis=1)
wnew

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,taxon_b,taxon_b_parent,taxon_b_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals
1,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals
2,taxo_data_images/Squirrel_posing.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,Q281124,Sciurus,Q2294121,genus of mammals,Q2294121,Sciurini,Q53106,tribe of mammals
3,taxo_data_images/Squirrel_posing.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q18215502,wikimedia,Q18215502,Sciurus,Q281124,subgenus of mammals,Q281124,Sciurus,Q2294121,genus of mammals
4,taxo_data_images/Ekorre_8218.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,Q281124,Sciurus,Q2294121,genus of mammals,Q2294121,Sciurini,Q53106,tribe of mammals
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343597,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q756,genus of plants,Q756,Plantae,Q19088,multicellular eukaryote of the kingdom Plantae
343598,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q756,genus of plants,Q756,Plantae,Q879246,multicellular eukaryote of the kingdom Plantae
343599,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants,Q5690336,Campanuloideae,Q155802,subfamily of plants
343600,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,Cercopidae,Q137213,family of insects


In [18]:
wnew = wnew.loc[~((wnew['filename'].duplicated(False)) & (wnew['taxon_b'].isna()))]

In [19]:
wnew_2 = pd.merge(wnew, non_df, left_on='taxon_b_parent', right_on='id', how='left').rename({'taxon': 'taxon_c', 'caption': 'taxon_c_caption', 'parent_taxon': 'taxon_c_parent'}, axis=1)
wnew_2

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,taxon_b,taxon_b_parent,taxon_b_caption,id,taxon_c,taxon_c_parent,taxon_c_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans
2,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals,Q25265,Felidae,Q10281053,family of mammals in the order of carnivorans
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals,Q25265,Felidae,Q4687717,family of mammals in the order of carnivorans
4,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,Pantherinae,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624489,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants,Q5690336,Campanuloideae,Q155802,subfamily of plants,Q155802,Campanulaceae,Q99461237,family of plants
624490,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants,Q5690336,Campanuloideae,Q155802,subfamily of plants,Q155802,Campanulaceae,Q100250955,family of plants
624491,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,Cercopidae,Q137213,family of insects,Q137213,Cercopoidea,Q634988,superfamily of insects
624492,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,Crabroninae,Q133302,subfamily of insects,Q133302,Crabronidae,Q531656,family of insects


In [20]:
wnew_2 = wnew_2.loc[~((wnew_2['filename'].duplicated(False)) & (wnew_2['taxon_c'].isna()))]

In [21]:
wnew_2 = wnew_2.loc[~wnew_2.duplicated()]

In [22]:
wnew_3 = pd.merge(wnew_2, non_df, left_on='taxon_c_parent', right_on='id', how='left').rename({'taxon': 'taxon_d', 'caption': 'taxon_d_caption', 'parent_taxon': 'taxon_d_parent'}, axis=1)
wnew_3

C:\Users\james\AppData\Local\Temp\ipykernel_28748\2249845503.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  wnew_3 = pd.merge(wnew_2, non_df, left_on='taxon_c_parent', right_on='id', how='left').rename({'taxon': 'taxon_d', 'caption': 'taxon_d_caption', 'parent_taxon': 'taxon_d_parent'}, axis=1)


,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_b_parent,taxon_b_caption,id_x,taxon_c,taxon_c_parent,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
2,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q10281053,family of mammals in the order of carnivorans,Q10281053,Feloidea,Q27070,superfamily of mammals
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q4687717,family of mammals in the order of carnivorans,Q4687717,Aeluroidea,Q27070,superfamily of mammals in the order Carnivora
4,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062339,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants,Q5690336,...,Q155802,subfamily of plants,Q155802,Campanulaceae,Q100250955,family of plants,Q100250955,Campanulinae,Q21730,suborder of plants
1062340,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,Q137213,family of insects,Q137213,Cercopoidea,Q634988,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects
1062341,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q133302,subfamily of insects,Q133302,Crabronidae,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps
1062342,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q133302,subfamily of insects,Q133302,Crabronidae,Q324132,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects


In [23]:
wnew_3 = wnew_3.loc[~wnew_3.duplicated()]
wnew_3

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_b_parent,taxon_b_caption,id_x,taxon_c,taxon_c_parent,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
2,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q10281053,family of mammals in the order of carnivorans,Q10281053,Feloidea,Q27070,superfamily of mammals
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q4687717,family of mammals in the order of carnivorans,Q4687717,Aeluroidea,Q27070,superfamily of mammals in the order Carnivora
4,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
6,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q10281053,family of mammals in the order of carnivorans,Q10281053,Feloidea,Q27070,superfamily of mammals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062339,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants,Q5690336,...,Q155802,subfamily of plants,Q155802,Campanulaceae,Q100250955,family of plants,Q100250955,Campanulinae,Q21730,suborder of plants
1062340,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,Q137213,family of insects,Q137213,Cercopoidea,Q634988,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects
1062341,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q133302,subfamily of insects,Q133302,Crabronidae,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps
1062342,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q133302,subfamily of insects,Q133302,Crabronidae,Q324132,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects


In [24]:
wnew_3 = wnew_3.loc[wnew_3['taxon_c'].notna()]
wnew_3 = wnew_3.loc[~((wnew_3['filename'].duplicated(False)) & (wnew_3['taxon_d'].isna()))]
wnew_3

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_b_parent,taxon_b_caption,id_x,taxon_c,taxon_c_parent,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
2,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q10281053,family of mammals in the order of carnivorans,Q10281053,Feloidea,Q27070,superfamily of mammals
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q4687717,family of mammals in the order of carnivorans,Q4687717,Aeluroidea,Q27070,superfamily of mammals in the order Carnivora
4,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora
6,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q25265,subfamily of mammals,Q25265,Felidae,Q10281053,family of mammals in the order of carnivorans,Q10281053,Feloidea,Q27070,superfamily of mammals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062339,taxo_data_images/Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Q157069,Campanula,Q5690336,genus of plants,Q5690336,...,Q155802,subfamily of plants,Q155802,Campanulaceae,Q100250955,family of plants,Q100250955,Campanulinae,Q21730,suborder of plants
1062340,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,Q137213,family of insects,Q137213,Cercopoidea,Q634988,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects
1062341,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q133302,subfamily of insects,Q133302,Crabronidae,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps
1062342,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q133302,subfamily of insects,Q133302,Crabronidae,Q324132,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects


In [25]:
wnew_4 = pd.merge(wnew_3, non_df, left_on='taxon_d_parent', right_on='id', how='left').rename({'taxon': 'taxon_e', 'caption': 'taxon_e_caption', 'parent_taxon': 'taxon_e_parent'}, axis=1)
wnew_4

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_parent,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption
0,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q25833,order of mammals
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals
2,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q25833,order of mammals
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals
4,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q27379,order of mammals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689865,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,Q634988,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects,Q202890,Auchenorrhyncha,Q26371,suborder of insects
1689866,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q1251421,superfamily of insects
1689867,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q22864,superfamily of insects
1689868,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q324132,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects,Q1251421,Aculeata,Q22864,infraorder of insects


In [26]:
wnew_4 = wnew_4.loc[~((wnew_4['filename'].duplicated(False)) & (wnew_4['taxon_e'].isna()))]

In [27]:
wnew_4 = wnew_4[~wnew_4.duplicated(None, False)]
wnew_4

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_parent,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals
4,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q27379,order of mammals
10,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals
12,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,Q27070,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689865,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,Q634988,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects,Q202890,Auchenorrhyncha,Q26371,suborder of insects
1689866,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q1251421,superfamily of insects
1689867,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q531656,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q22864,superfamily of insects
1689868,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,Q324132,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects,Q1251421,Aculeata,Q22864,infraorder of insects


In [28]:
l = wnew_4[~wnew_4.duplicated('filename')]['taxon_e_caption'].map(lambda x: x.split(' ')[0] if type(x) is not float else None).value_counts()
l

order              32053
subclass           29950
class              25945
clade              20719
suborder           18856
                   ...  
animals                1
Lophotrochozoan        1
formerly               1
Unterreich             1
subdivsion             1
Name: taxon_e_caption, Length: 92, dtype: int64

In [29]:
non_df.loc[non_df.id=='Q764', 'parent_taxon'].values.tolist()

['Q7239', 'Q19088', 'Q2943473']

In [30]:
all_taxon_parent_left = wnew_4['taxon_e_parent'].unique()
all_taxon_parent_left

array(['Q20868', 'Q7377', 'Q27379', ..., 'Q21074684', 'Q131092',
       'Q131804'], dtype=object)

In [31]:
len(all_taxon_parent_left)

1079

In [32]:
o = list(map(find_animalia, all_taxon_parent_left))
o

-Q7377
Q150866
Q150866
Q10915
Q25241
Q3280581
Q84149
Q25241
Q25241
Q19159
Q26214
Q26214
Q26214
Q27207
Q134681
Q3059636
Q134681
Q27207
Q160830
Q1378800
Q1378800
Q160830
Q23809240
Q1209254
Q150598
Q1209254
Q19159
Q2082668
Q134683
Q181537
Q189069
Q21353733
Q21353733
Q189069
Q1349838
Q930410
Q5853182
Q322241
Q788221
Q132812
Q5414469
Q131801
Q131801
Q5382672
Q680936
Q950661
Q3323086
Q5852697
Q2082668
Q28746016
Q2082668
Q2082668
Q7377
Q7377
Q7377
Q7377
Q130942
Q2513125
Q26708524
Q2513125
Q2141770
Q2478975
Q2141770
Q766050
Q2910821
Q2497754
Q2910821
Q130942
Q130942
Q25833
Q17092469
Q25833
Q133342
Q132666
Q132666
Q27379
Q7439311
Q5444079
Q20868
Q2475989
Q3402333
Q20868
Q20868
-Q7377
Q7377
Q7377
Q7377
Q7377
-Q27379
-Q471797
Q92189203
Q660432
Q10850
Q10850
Q10850
-Q724477
--Q1390
Q105146
Q1633496
Q671280
Q671280
Q105146
Q1390
Q22708
Q1210274
Q22708
Q22708
Q1925093
Q23005
Q23005
Q5861654
Q304358
Q22671
Q37140800
Q22671
Q21215044
Q22671
Q2982449
Q22671
Q147877
--------------Q25306
Q25306
Q25306
Q2

[6,
 2,
 6,
 3,
 4,
 -1,
 8,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 4,
 30,
 9,
 8,
 10,
 9,
 7,
 5,
 6,
 17,
 6,
 6,
 14,
 22,
 21,
 6,
 -1,
 10,
 16,
 11,
 8,
 7,
 11,
 5,
 12,
 11,
 7,
 9,
 6,
 7,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 9,
 15,
 8,
 9,
 15,
 17,
 10,
 8,
 5,
 7,
 11,
 -1,
 -1,
 -1,
 18,
 17,
 8,
 8,
 7,
 7,
 3,
 4,
 8,
 6,
 5,
 1,
 7,
 5,
 10,
 9,
 -1,
 -1,
 6,
 4,
 7,
 4,
 7,
 9,
 7,
 8,
 7,
 7,
 -1,
 9,
 -1,
 9,
 8,
 3,
 9,
 9,
 13,
 8,
 5,
 7,
 5,
 9,
 9,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 16,
 8,
 10,
 5,
 -1,
 -1,
 -1,
 10,
 16,
 15,
 -1,
 11,
 16,
 19,
 -1,
 -1,
 -1,
 -1,
 -1,
 6,
 8,
 18,
 13,
 4,
 -1,
 10,
 11,
 10,
 -1,
 -1,
 -1,
 -1,
 -1,
 13,
 10,
 15,
 12,
 7,
 6,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 13,
 7,
 -1,
 11,
 -1,
 -1,
 -1,
 6,
 5,
 -1,
 3,
 5,
 8,
 5,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 8,
 11,
 13,
 13,
 -1,
 19,
 17,
 -1,
 -1,

In [33]:
from collections import Counter
Counter(o)

Counter({6: 74,
         2: 9,
         3: 36,
         4: 54,
         -1: 421,
         8: 68,
         30: 1,
         9: 80,
         10: 65,
         7: 71,
         5: 60,
         17: 7,
         14: 8,
         22: 2,
         21: 3,
         16: 9,
         11: 42,
         12: 23,
         15: 6,
         18: 9,
         1: 1,
         13: 19,
         19: 5,
         31: 1,
         20: 1,
         23: 2,
         25: 1,
         27: 1})

In [34]:
all_taxon_parent_left[o.index(23)]

'Q940396'

In [35]:
lookahead = dict(zip(all_taxon_parent_left, o))
lookahead

{'Q20868': 6,
 'Q7377': 2,
 'Q27379': 6,
 'Q10850': 3,
 'Q724477': 4,
 'Q92190920': -1,
 'Q147877': 8,
 'Q8316': -1,
 'Q1307404': -1,
 'Q1372246': -1,
 'Q14562931': -1,
 'Q869087': -1,
 'Q23905211': -1,
 'Q14832431': -1,
 'Q25814': -1,
 'Q13582382': -1,
 'Q3442619': -1,
 'Q338878': -1,
 'Q2287128': -1,
 'Q100233864': -1,
 'Q27414': 4,
 'Q938944': 30,
 'Q21590143': 9,
 'Q2330918': 8,
 'Q19595935': 10,
 'Q19598418': 9,
 'Q19168': 7,
 'Q25241': 5,
 'Q19159': 6,
 'Q136586': 17,
 'Q5113': 6,
 'Q72629': 6,
 'Q25341': 14,
 'Q25537662': 22,
 'Q16986904': 21,
 'Q84149': 6,
 'Q777371': -1,
 'Q1209254': 10,
 'Q4828939': 16,
 'Q1949508': 11,
 'Q131060': 8,
 'Q22671': 7,
 'Q27732132': 11,
 'Q182978': 5,
 'Q22109111': 12,
 'Q660475': 11,
 'Q127282': 7,
 'Q204861': 9,
 'Q26214': 6,
 'Q27207': 7,
 'Q13647779': -1,
 'Q14467364': -1,
 'Q14942319': -1,
 'Q2683213': -1,
 'Q165468': -1,
 'Q1259422': -1,
 'Q4006723': -1,
 'Q2070446': -1,
 'Q23927181': -1,
 'Q174698': -1,
 'Q22007593': 9,
 'Q194240': 15,
 'Q

In [36]:
wnew_4['lookahead'] = wnew_4['taxon_e_parent'].map(lambda x: lookahead.get(x, False) if type(x) is not float else False)

In [37]:
wnew_4[wnew_4['parent_taxon_x']=="Q3506745"]

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption,lookahead
104,taxo_data_images/Gharial_san_diego.jpg,Gavialis gangeticus,['This is an example of species Gavialis gange...,Q3506745,wikimedia,Q3506745,Gavialis,Q878388,genus of reptiles,Q878388,...,superfamily of large reptiles,Q107389342,Longirostres,Q25363,clade of crocodilians,Q25363,Crocodilia,Q938944,order of large reptiles,30
105,taxo_data_images/Gharial_(Gavialis_gangeticus)...,Gavialis gangeticus,['This is an example of species Gavialis gange...,Q3506745,wikimedia,Q3506745,Gavialis,Q878388,genus of reptiles,Q878388,...,superfamily of large reptiles,Q107389342,Longirostres,Q25363,clade of crocodilians,Q25363,Crocodilia,Q938944,order of large reptiles,30
106,taxo_data_images/Gharial_(Gavialis_gangeticus)...,Gavialis gangeticus,['This is an example of species Gavialis gange...,Q3506745,wikimedia,Q3506745,Gavialis,Q878388,genus of reptiles,Q878388,...,superfamily of large reptiles,Q107389342,Longirostres,Q25363,clade of crocodilians,Q25363,Crocodilia,Q938944,order of large reptiles,30


In [38]:
backup = wnew_4[(((wnew_4[['genus']+[x for x in wnew_4.columns if x[:6]=='taxon_' and len(x)==7]]=='Animalia').sum(axis=1) != 0) \
| (wnew_4['taxon_e_parent'].map(lambda x: lookahead.get(x, -1) if type(x) is not float else -1) > -1))].copy()
wnew_4 = backup

In [39]:
backup

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption,lookahead
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals,6
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals,2
4,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q27379,order of mammals,6
10,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals,6
12,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689865,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects,Q202890,Auchenorrhyncha,Q26371,suborder of insects,9
1689866,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q1251421,superfamily of insects,10
1689867,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q22864,superfamily of insects,9
1689868,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects,Q1251421,Aculeata,Q22864,infraorder of insects,9


In [40]:
tol = 10

In [41]:
animalia = wnew_4.copy()
animalia['lookahead'] = animalia['taxon_e_parent'].map(lambda x: lookahead.get(x, -1) if type(x) is not float else -1)
animalia['lookahead'] = animalia['lookahead'].map(lambda x: 100000 if x==-1 else x)
animalia['lookahead'] = animalia['lookahead'].map(lambda x: 99999 if x<tol and x > -1 else x)
animalia

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption,lookahead
1,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals,99999
3,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals,99999
4,taxo_data_images/Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q27379,order of mammals,99999
10,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q20868,order of mammals,99999
12,taxo_data_images/Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Q127960,Panthera,Q230177,genus of mammals,Q230177,...,family of mammals in the order of carnivorans,Q27070,Feliformia,Q25306,suborder of mammals in the order Carnivora,Q25306,Carnivora,Q7377,order of mammals,99999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689865,taxo_data_images/Cercopis_septemmaculata_(DSCN...,Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Q2946582,Cercopis,Q886877,genus of insects,Q886877,...,superfamily of insects,Q634988,Cicadomorpha,Q202890,infraorder of insects,Q202890,Auchenorrhyncha,Q26371,suborder of insects,99999
1689866,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q1251421,superfamily of insects,10
1689867,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,family of insects,Q531656,Spheciformes,Q324132,lineage of wasps,Q324132,Apoidea,Q22864,superfamily of insects,99999
1689868,taxo_data_images/Nitela_philippinica_02.jpg,Nitela philippinica,['This is an example of species Nitela philipp...,Q1264163,wikimedia,Q1264163,Nitela,Q148664,genus of insects,Q148664,...,family of insects,Q324132,Apoidea,Q1251421,superfamily of insects,Q1251421,Aculeata,Q22864,infraorder of insects,99999


In [42]:
animalia[animalia['species']=='Amphiprion ephippium'].iloc[:,:40]

,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption,lookahead
1517377,taxo_data_images/AmphiprionEphippium.png,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,vertebrate animal that lives in water and (typ...,Q26214,Gnathostomata,Q25241,infraphylum of jawed vertebrates,Q25241,Vertebrata,Q84149,subphylum of chordates,99999
1517379,taxo_data_images/AmphiprionEphippium.png,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,vertebrate animal that lives in water and (typ...,Q26214,Gnathostomata,Q3280581,infraphylum of jawed vertebrates,Q3280581,Olfactores,Q10915,clade of chordates,99999
1517380,taxo_data_images/AmphiprionEphippium.png,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,vertebrate animal that lives in water and (typ...,Q26214,Gnathostomata,Q84149,infraphylum of jawed vertebrates,Q84149,Craniata,Q3280581,clade of chordates,99999
1517381,taxo_data_images/AmphiprionEphippium.png,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,family of fishes,Q840552,Labroidei,Q127595,suborder of fishes,Q127595,Perciformes,Q660475,order of fishes,11
1517382,taxo_data_images/AmphiprionEphippium.png,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,family of fishes,Q840552,Labroidei,Q127595,suborder of fishes,Q127595,Perciformes,Q30026706,order of fishes,13
1517383,taxo_data_images/AmphiprionEphippium.png,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,family of fishes,Q840552,Labroidei,Q127595,suborder of fishes,Q127595,Perciformes,Q343460,order of fishes,99999
1517385,taxo_data_images/20171101_Amphiprion_ephippium...,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,vertebrate animal that lives in water and (typ...,Q26214,Gnathostomata,Q25241,infraphylum of jawed vertebrates,Q25241,Vertebrata,Q84149,subphylum of chordates,99999
1517387,taxo_data_images/20171101_Amphiprion_ephippium...,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,vertebrate animal that lives in water and (typ...,Q26214,Gnathostomata,Q3280581,infraphylum of jawed vertebrates,Q3280581,Olfactores,Q10915,clade of chordates,99999
1517388,taxo_data_images/20171101_Amphiprion_ephippium...,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,vertebrate animal that lives in water and (typ...,Q26214,Gnathostomata,Q84149,infraphylum of jawed vertebrates,Q84149,Craniata,Q3280581,clade of chordates,99999
1517389,taxo_data_images/20171101_Amphiprion_ephippium...,Amphiprion ephippium,['This is an example of species Amphiprion eph...,Q6193348,wikimedia,Q6193348,Amphiprion,Q472616,genus of clownfish,Q472616,...,family of fishes,Q840552,Labroidei,Q127595,suborder of fishes,Q127595,Perciformes,Q660475,order of fishes,11


In [43]:
animalia = animalia.sort_values(['filename','lookahead', 'taxon_e']).drop_duplicates('filename', 'first')
animalia

C:\Users\james\AppData\Local\Temp\ipykernel_28748\4010397358.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop_duplicates except for the argument 'subset' will be keyword-only
  animalia = animalia.sort_values(['filename','lookahead', 'taxon_e']).drop_duplicates('filename', 'first')


,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,id_y,...,taxon_c_caption,id_y,taxon_d,taxon_d_parent,taxon_d_caption,id,taxon_e,taxon_e_parent,taxon_e_caption,lookahead
501114,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,Q47123120,...,family of molluscs,Q4673569,Achatinoidea,Q57262973,superfamily of molluscs,Q57262973,Achatinina,Q972302,suborder of molluscs,10
104502,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,Q592684,...,order of insects,Q2087279,Dictyoptera,Q23005,superorder of insects,Q23005,Neoptera,Q1925093,infraclass of insects,99999
430070,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,Q914293,...,family of rodents,Q1063601,Hystricognathi,Q15055130,infraorder of mammals,Q15055130,Hystricognatha,Q10850,suborder of mammals,99999
858984,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,Q615799,...,superfamily of beetles,Q692384,Cucujiformia,Q147877,infraorder of beetles,Q147877,Polyphaga,Q22671,suborder of beetles,99999
915133,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,['This is an example of species Vespa affinis....,Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,Q1076176,...,family of insects,Q425636,Vespoidea,Q1251421,superfamily of insects,Q1251421,Aculeata,Q22864,infraorder of insects,99999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526250,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,Q25420,...,superfamily of beetles,Q374779,Elateriformia,Q147877,infraorder of beetles,Q147877,Polyphaga,Q22671,suborder of beetles,99999
1536203,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,['This is an example of species Acheilognathus...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,Q35047,...,order of fishes,Q149081,Ostariophysi,Q22107399,superorder of fishes,Q22107399,Otomorpha,Q1103396,cohort of fishes,10
1326106,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,['This is an example of species Agelastica coe...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,Q2842229,...,subfamily of leaf beetles,Q193407,Chrysomelidae,Q920316,family of beetles,Q920316,Chrysomeloidea,Q692384,superfamily of beetles,99999
84120,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,['This is an example of species Takydromus wol...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,Q649583,...,family of reptiles,Q13518423,Lacertoidea,Q13518421,clade of reptiles,Q13518421,Episquamata,Q122422,taxon of reptiles,18


In [44]:
def get_chr(num):
    if num <= 122:
        return chr(num)
    div = (num-97)//26
    mod = (num-97)%26
    return chr(97+div-1) + chr(97+mod)

In [45]:
len(animalia)

137706

In [46]:
start = 101

for i in range(31):
    print(i, len(animalia))
    animalia = pd.merge(animalia, non_df, left_on='taxon_'+get_chr(start+i)+'_parent', right_on='id', how='left').rename({'taxon': 'taxon_'+get_chr(start+1+i), 'caption': 'taxon_'+get_chr(start+1+i)+'_caption', 'parent_taxon': 'taxon_'+get_chr(start+1+i)+'_parent'}, axis=1)
    animalia['lookahead'] = animalia['taxon_'+get_chr(start+i+1)+'_parent'].map(lambda x: lookahead.get(x, -1) if type(x) is not float else -1)
    animalia['lookahead'] = animalia['lookahead'].map(lambda x: 100000 if x==-1 else x)
    if i<tol:
        animalia['lookahead'] = animalia['lookahead'].map(lambda x: 99999 if x<(tol-i) else x)
    animalia = animalia.sort_values(['filename','lookahead', 'taxon_e']).drop_duplicates('filename', 'first')

0 137706


C:\Users\james\AppData\Local\Temp\ipykernel_28748\756250547.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  animalia = pd.merge(animalia, non_df, left_on='taxon_'+get_chr(start+i)+'_parent', right_on='id', how='left').rename({'taxon': 'taxon_'+get_chr(start+1+i), 'caption': 'taxon_'+get_chr(start+1+i)+'_caption', 'parent_taxon': 'taxon_'+get_chr(start+1+i)+'_parent'}, axis=1)
C:\Users\james\AppData\Local\Temp\ipykernel_28748\756250547.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop_duplicates except for the argument 'subset' will be keyword-only
  animalia = animalia.sort_values(['filename','lookahead', 'taxon_e']).drop_duplicates('filename', 'first')


1 137706
2 137706
3 137706
4 137706
5 137706
6 137706
7 137706
8 137706
9 137706
10 137706
11 137706
12 137706
13 137706
14 137706
15 137706
16 137706
17 137706
18 137706
19 137706
20 137706
21 137706
22 137706
23 137706
24 137706
25 137706
26 137706
27 137706
28 137706
29 137706
30 137706


In [47]:
animalia = animalia[(animalia[['genus']+[x for x in animalia.columns if x[:6]=='taxon_' and len(x)<10]]=='Animalia').sum(axis=1)==1]

In [48]:
animalia[[col for col in animalia.columns if col.find('caption')>0]].apply(lambda x: list(filter(lambda x: type(x) is not float, x.to_list()))[::-1], axis=1).map(lambda x: x[x.index('kingdom of multicellular eukaryotic organisms'):])

0         [kingdom of multicellular eukaryotic organisms...
1         [kingdom of multicellular eukaryotic organisms...
2         [kingdom of multicellular eukaryotic organisms...
3         [kingdom of multicellular eukaryotic organisms...
4         [kingdom of multicellular eukaryotic organisms...
                                ...                        
137798    [kingdom of multicellular eukaryotic organisms...
137799    [kingdom of multicellular eukaryotic organisms...
137800    [kingdom of multicellular eukaryotic organisms...
137801    [kingdom of multicellular eukaryotic organisms...
137802    [kingdom of multicellular eukaryotic organisms...
Length: 137705, dtype: object

In [49]:
x = (animalia[['genus']+[col for col in animalia.columns if len(col)<10 and col[:5]=='taxon']].apply(lambda x: list(filter(lambda x: type(x) is not float, x.to_list()))[::-1], axis=1)).map(lambda x: x[x.index('Animalia'):]).to_list()
y = (animalia[[col for col in animalia.columns if col.find('caption')>0]].apply(lambda x: list(filter(lambda x: type(x) is not float, x.to_list()))[::-1], axis=1)).map(lambda x: x[x.index('kingdom of multicellular eukaryotic organisms'):]).to_list()

In [50]:
todrop = ['genus']+[col for col in wnew_4.columns if len(col)<10 and col[:5]=='taxon'] + [col for col in wnew_4.columns if col.find('caption')>0 or col.find('parent')!=-1 or col.find('id')!=-1]

In [51]:
zz = list(map(len, x))
strict_animalia = animalia.reset_index()[pd.Series(zz) >= 7]
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ah_parent,taxon_ah_caption,id_x,taxon_ai,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,['This is an example of species Vespa affinis....,Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,['This is an example of species Acheilognathus...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,['This is an example of species Agelastica coe...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137703,137801,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,['This is an example of species Takydromus wol...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
strict_animalia[~strict_animalia.drop(['filename', 'index'], axis=1).duplicated(None, False) & strict_animalia.duplicated('species', False)]

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ah_parent,taxon_ah_caption,id_x,taxon_ai,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption
1249,1253,taxo_data_images/2019_05_18_Chrysocrambus_crat...,Chrysocrambus craterella,['This is an example of species Chrysocrambus ...,Q283890,wikimedia,Q283890,Chrysocrambus,Q132980,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5403,5409,taxo_data_images/Aglais_io_IMG_4116.JPG,Aglais io,"['This is an example of species Aglais io.', '...",Q32469,wikimedia,Q32469,Aglais,Q32476,genus of brush-footed butterflies,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30023,30053,taxo_data_images/Chrysocrambus.craterellus.mou...,Chrysocrambus craterella,['This is an example of species Chrysocrambus ...,Q283890,wikimedia,Q283890,Chrysocrambus,Q132980,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53676,53734,taxo_data_images/Galapagos_Geochelone_nigra_po...,Chelonoidis niger,['This is an example of species Chelonoidis ni...,Q2080003,wikimedia,Q2080003,Chelonoidis,Q46360,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64958,65016,taxo_data_images/Inachis_io_Lill-Jansskogen.JPG,Aglais io,"['This is an example of species Aglais io.', '...",Q15690624,wikimedia,Q15690624,Inachis,Q32476,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88730,88808,taxo_data_images/Nippononeta_alpina_(10.3897-z...,Nippononeta alpina,['This is an example of species Nippononeta al...,Q2226236,wikimedia,Q2226236,Nippononeta,Q10589,genus of arachnids,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# strict_animalia.iloc[5404,4:] = strict_animalia.iloc[64958,4:]

In [54]:
strict_animalia[strict_animalia.species=='Chelonoidis niger'].iloc[1,3]

"['This is an example of species Chelonoidis niger.', 'Image of an animal with taxon name: Chelonoidis niger.', 'Species: Chelonoidis niger. Chelonoidis species complex in the Galápagos Islands; species with many subspecies', 'Chelonoidis species complex in the Galápagos Islands; species with many subspecies', 'Chelonoidis species complex in the Galápagos Islands; species with many subspecies. Taxon Name: Chelonoidis nigerChelonoidis species complex in the Galápagos Islands; species with many subspecies. Species: Chelonoidis niger']"

In [55]:
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ah_parent,taxon_ah_caption,id_x,taxon_ai,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,['This is an example of species Vespa affinis....,Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,['This is an example of species Acheilognathus...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,['This is an example of species Agelastica coe...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137703,137801,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,['This is an example of species Takydromus wol...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# backup_taxons = strict_animalia[['filename', 'species', 'genus']+[col for col in strict_animalia.columns if col[:5]=='taxon']].copy(deep=True)
# backup_taxons['filename'] = backup_taxons['filename'].str[17:]
# backup_taxons.to_csv('filename_taxons_taxdesc_df.csv', index=False, encoding='utf-8')

In [57]:
x = (strict_animalia[['genus']+[col for col in strict_animalia.columns if len(col)<10 and col[:5]=='taxon']].apply(lambda x: list(filter(lambda x: type(x) is not float, x.to_list()))[::-1], axis=1)).map(lambda x: x[x.index('Animalia'):]).to_list()
y = (strict_animalia[[col for col in strict_animalia.columns if col.find('caption')>0]].apply(lambda x: list(filter(lambda x: type(x) is not float, x.to_list()))[::-1], axis=1)).map(lambda x: x[x.index('kingdom of multicellular eukaryotic organisms'):]).to_list()

In [58]:
keywords = ['kingdom', 'phylum', 'order', 'class', 'family', 'genus']

In [59]:
import re
MATCHER = re.compile('\s?(kingdom)|(phylum)|(\S*phylum)|(\s?class)|(\S*class)|(\s?order)|(\S*order)|(\s?family)|(\S*family)|(\S*genus)')
STRICT_MATCHER = re.compile('(kingdom)|(phylum)|(class)|(order)|(family)|(genus)')

In [60]:
vkeys = ["kingdom","phylum","class","order","family","genus"]
impp = [{MATCHER.search(ent).group():(z,ent) for z,ent in zip(oo,li) if MATCHER.search(ent)} for oo,li in zip(x,y)]

In [71]:
to_test_df = [{v:entry.get(v) for v in vkeys} for entry in impp]

In [61]:
mmm = list(map(lambda x: sum([1 for v in vkeys if x.get(v) is not None]), impp))

In [62]:
Counter(mmm)

Counter({5: 18803, 6: 117862, 4: 873, 3: 49, 2: 6})

In [63]:
assert False

AssertionError: 

In [ ]:
indices = [i for i,m in enumerate(mmm) if m==4]
for i in indices:
    print(y[i])

['kingdom of multicellular eukaryotic organisms', 'animal without a vertebral column', 'Phylum of animals, flatworms', 'class of worms', 'subclass of worms', 'family of worms', 'genus of worms']
['kingdom of multicellular eukaryotic organisms', 'animal without a vertebral column', 'phylum of invertebrates with jointed exoskeletons', 'subphylum of arthropods', 'class of crustaceans', 'subclass of crustaceans containing almost all living malacostracans', 'group of malacostracan crustaceans', 'crustacean order', 'crustacean family', 'genus of arthropods']
['kingdom of multicellular eukaryotic organisms', 'animal without a vertebral column', 'phylum of invertebrates with jointed exoskeletons', 'subphylum of arthropods', 'class of crustaceans', 'subclass of crustaceans containing almost all living malacostracans', 'group of malacostracan crustaceans', 'crustacean order', 'crustacean family', 'genus of arthropods']
['kingdom of multicellular eukaryotic organisms', 'class of tetrapods', 'subc

In [72]:
strict_animalia['match_classes'] = impp
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ah_caption,id_x,taxon_ai,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,['This is an example of species Vespa affinis....,Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,['This is an example of species Acheilognathus...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,['This is an example of species Agelastica coe...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."
137703,137801,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,['This is an example of species Takydromus wol...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell..."


In [73]:
strict_animalia['match_classes'] = strict_animalia['match_classes'].map(lambda x: {k:x.get(k) for k in vkeys if x.get(k) is not None})
strict_animalia['match_classes']

0         {'kingdom': ('Animalia', 'kingdom of multicell...
1         {'kingdom': ('Animalia', 'kingdom of multicell...
2         {'kingdom': ('Animalia', 'kingdom of multicell...
3         {'kingdom': ('Animalia', 'kingdom of multicell...
4         {'kingdom': ('Animalia', 'kingdom of multicell...
                                ...                        
137700    {'kingdom': ('Animalia', 'kingdom of multicell...
137701    {'kingdom': ('Animalia', 'kingdom of multicell...
137702    {'kingdom': ('Animalia', 'kingdom of multicell...
137703    {'kingdom': ('Animalia', 'kingdom of multicell...
137704    {'kingdom': ('Animalia', 'kingdom of multicell...
Name: match_classes, Length: 137593, dtype: object

In [195]:
strict_animalia['match_classes'].iloc[0]

{'kingdom': ('Animalia', 'kingdom of multicellular eukaryotic organisms'),
 'phylum': ('Mollusca', 'large phylum of invertebrate animals'),
 'class': ('Gastropoda', 'class of molluscs'),
 'family': ('Achatinidae', 'family of molluscs'),
 'genus': ('Allopeas', 'genus of molluscs')}

In [205]:
for col in vkeys:
    strict_animalia[col] = strict_animalia.apply(lambda x: ", ".join(x['match_classes'].get(col)) if type(x['match_classes'].get(col)) is tuple else None, axis=1)
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes,kingdom,phylum,class,order,family
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Q3738109,"Allopeas, genus of molluscs",Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Mollusca, large phylum of invertebrate animals","Gastropoda, class of molluscs",None,"Achatinidae, family of molluscs"
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Q10607011,"Odontomantis, genus of mantises",Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Mantodea, order of insects","Hymenopodidae, family of insects"
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Q286088,"Cavia, genus of mammals",Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...",None,"Mammalia, class of tetrapods","Rodentia, order of mammals","Caviidae, family of rodents"
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Q3518056,"Tenebrio, genus of insects",Q615799,genus of insects,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Coleoptera, order of insects","Tenebrionidae, family of beetles"
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,['This is an example of species Vespa affinis....,Q334422,wikimedia,Q334422,"Vespa, genus of insects",Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Hymenoptera, order of insects","Vespidae, family of insects"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,Q7733181,"Pyrocoelia, genus of insects",Q25420,genus of insects,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Coleoptera, order of insects","Lampyridae, family of beetles"
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,['This is an example of species Acheilognathus...,Q481845,wikimedia,Q481845,"Acheilognathus, genus of fishes",Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Chordata, phylum of animals","Actinopterygii, class of ray-finned bony fishes","Cypriniformes, order of fishes","Cyprinidae, family of fishes"
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,['This is an example of species Agelastica coe...,Q1941116,wikimedia,Q1941116,"Agelastica, genus of leaf beetles",Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Coleoptera, order of insects","Chrysomelidae, family of beetl

In [209]:
strict_animalia[vkeys].isna().sum()

kingdom       0
phylum     6012
class      5557
order      5362
family     2489
genus      1300
dtype: int64

In [169]:
match_classes = match_classes.apply(lambda x: [", ".join(y) if type(y) is tuple else None for y in x] if x is not None else x, axis=1)
match_classes

0         [Animalia, kingdom of multicellular eukaryotic...
1         [Animalia, kingdom of multicellular eukaryotic...
2         [Animalia, kingdom of multicellular eukaryotic...
3         [Animalia, kingdom of multicellular eukaryotic...
4         [Animalia, kingdom of multicellular eukaryotic...
                                ...                        
137700    [Animalia, kingdom of multicellular eukaryotic...
137701    [Animalia, kingdom of multicellular eukaryotic...
137702    [Animalia, kingdom of multicellular eukaryotic...
137703    [Animalia, kingdom of multicellular eukaryotic...
137704    [Animalia, kingdom of multicellular eukaryotic...
Length: 137593, dtype: object

In [184]:
newdf = pd.DataFrame()
newdf[["kingdom", "phylum", "class", "family", "genus", "order"]] = match_classes.to_list()
newdf

,kingdom,phylum,class,family,genus,order
0,"Animalia, kingdom of multicellular eukaryotic ...","Mollusca, large phylum of invertebrate animals","Gastropoda, class of molluscs","Achatinidae, family of molluscs","Allopeas, genus of molluscs",None
1,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Hymenopodidae, family of insects","Odontomantis, genus of mantises","Mantodea, order of insects"
2,"Animalia, kingdom of multicellular eukaryotic ...",None,"Mammalia, class of tetrapods","Caviidae, family of rodents","Cavia, genus of mammals","Rodentia, order of mammals"
3,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Tenebrionidae, family of beetles","Tenebrio, genus of insects","Coleoptera, order of insects"
4,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Vespidae, family of insects","Vespa, genus of insects","Hymenoptera, order of insects"
...,...,...,...,...,...,...
137588,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Lampyridae, family of beetles","Pyrocoelia, genus of insects","Coleoptera, order of insects"
137589,"Animalia, kingdom of multicellular eukaryotic ...","Chordata, phylum of animals","Actinopterygii, class of ray-finned bony fishes","Cyprinidae, family of fishes","Acheilognathus, genus of fishes","Cypriniformes, order of fishes"
137590,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Chrysomelidae, family of beetles","Agelastica, genus of leaf beetles","Coleoptera, order of insects"
137591,"Animalia, kingdom of multicellular eukaryotic ...","Chordata, phylum of animals",None,"Lacertidae, family of reptiles","Takydromus, genus of reptiles","Squamata, order of reptiles"


In [219]:
newdf['species'] = strict_animalia.reset_index().drop(['level_0', 'index'], axis=1)['species']

In [221]:
newdf.isna().sum()

kingdom       0
phylum     6012
class      5557
family     2489
genus      1300
order      5362
species       0
dtype: int64

In [251]:
newdf[newdf.duplicated('species', False) & ~newdf.duplicated(None, False)].sort_values(['species'] + vkeys)

,kingdom,phylum,class,family,genus,order,species
5402,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Nymphalidae, family of insects","Aglais, genus of brush-footed butterflies","Lepidoptera, order of insects that includes bu...",Aglais io
64899,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Nymphalidae, family of insects","Inachis, genus of insects","Lepidoptera, order of insects that includes bu...",Aglais io


In [255]:
newdf = newdf.drop(64899, axis=0)
newdf = newdf.drop_duplicates()
newdf

,kingdom,phylum,class,family,genus,order,species
0,"Animalia, kingdom of multicellular eukaryotic ...","Mollusca, large phylum of invertebrate animals","Gastropoda, class of molluscs","Achatinidae, family of molluscs","Allopeas, genus of molluscs",None,Allopeas clavulinum
1,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Hymenopodidae, family of insects","Odontomantis, genus of mantises","Mantodea, order of insects",Odontomantis planiceps
2,"Animalia, kingdom of multicellular eukaryotic ...",None,"Mammalia, class of tetrapods","Caviidae, family of rodents","Cavia, genus of mammals","Rodentia, order of mammals",Cavia porcellus
3,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Tenebrionidae, family of beetles","Tenebrio, genus of insects","Coleoptera, order of insects",Tenebrio molitor
4,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Vespidae, family of insects","Vespa, genus of insects","Hymenoptera, order of insects",Vespa affinis
...,...,...,...,...,...,...,...
137587,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Carabidae, family of beetles","Carabus, genus of insects","Coleoptera, order of insects",Carabus mirabilissimus
137588,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Lampyridae, family of beetles","Pyrocoelia, genus of insects","Coleoptera, order of insects",Pyrocoelia rufa
137589,"Animalia, kingdom of multicellular eukaryotic ...","Chordata, phylum of animals","Actinopterygii, class of ray-finned bony fishes","Cyprinidae, family of fishes","Acheilognathus, genus of fishes","Cypriniformes, order of fishes",Acheilognathus hondae
137590,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Chrysomelidae, family of beetles","Agelastica, genus of leaf beetles","Coleoptera, order of insects",Agelastica coerulea


In [261]:
newdf.loc[newdf.kingdom=="Eumetazoa, subkingdom of animals", "kingdom"] = "Animalia, kingdom of multicellular eukaryotic organisms"
newdf

,kingdom,phylum,class,family,genus,order,species
0,"Animalia, kingdom of multicellular eukaryotic ...","Mollusca, large phylum of invertebrate animals","Gastropoda, class of molluscs","Achatinidae, family of molluscs","Allopeas, genus of molluscs",None,Allopeas clavulinum
1,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Hymenopodidae, family of insects","Odontomantis, genus of mantises","Mantodea, order of insects",Odontomantis planiceps
2,"Animalia, kingdom of multicellular eukaryotic ...",None,"Mammalia, class of tetrapods","Caviidae, family of rodents","Cavia, genus of mammals","Rodentia, order of mammals",Cavia porcellus
3,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Tenebrionidae, family of beetles","Tenebrio, genus of insects","Coleoptera, order of insects",Tenebrio molitor
4,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Vespidae, family of insects","Vespa, genus of insects","Hymenoptera, order of insects",Vespa affinis
...,...,...,...,...,...,...,...
137587,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Carabidae, family of beetles","Carabus, genus of insects","Coleoptera, order of insects",Carabus mirabilissimus
137588,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Lampyridae, family of beetles","Pyrocoelia, genus of insects","Coleoptera, order of insects",Pyrocoelia rufa
137589,"Animalia, kingdom of multicellular eukaryotic ...","Chordata, phylum of animals","Actinopterygii, class of ray-finned bony fishes","Cyprinidae, family of fishes","Acheilognathus, genus of fishes","Cypriniformes, order of fishes",Acheilognathus hondae
137590,"Animalia, kingdom of multicellular eukaryotic ...","Arthropoda, phylum of invertebrates with joint...","Insecta, class of arthropods","Chrysomelidae, family of beetles","Agelastica, genus of leaf beetles","Coleoptera, order of insects",Agelastica coerulea


In [265]:
for col in newdf:
    print(col, len(newdf[col].unique()))

kingdom 1
phylum 30
class 97
family 4387
genus 36533
order 579
species 127784


In [268]:
newdf.to_csv("test_hierarchy_reference.csv", encoding="utf-8", index=False)

In [237]:
strict_animalia.iloc[50221]

index                                                         50326
filename               taxo_data_images/Extatosoma_tiaratum_119.JPG
species                                          giant stick insect
captions          ['This is an example of species giant stick in...
parent_taxon_x                                             Q1384915
                                        ...                        
kingdom           Animalia, kingdom of multicellular eukaryotic ...
phylum            Arthropoda, phylum of invertebrates with joint...
class                                  Insecta, class of arthropods
order                     Phasmida, order of stick and leaf insects
family                               Phasmatidae, family of insects
Name: 50268, Length: 157, dtype: object

In [ ]:
strict_animalia['match_classes'].iloc[0]

{'kingdom': ('Animalia', 'kingdom of multicellular eukaryotic organisms'),
 'phylum': ('Mollusca', 'large phylum of invertebrate animals'),
 'class': ('Gastropoda', 'class of molluscs'),
 'family': ('Achatinidae', 'family of molluscs'),
 'genus': ('Allopeas', 'genus of molluscs')}

In [ ]:
strict_animalia['extra_captions'] = strict_animalia['match_classes'].map(lambda x: [', '.join(y) for y in x.values()])
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,id_x,taxon_ai,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes,extra_captions
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,['This is an example of species Vespa affinis....,Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,['This is an example of species Acheilognathus...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,['This is an example of species Agelastica coe...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."
137703,137801,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,['This is an example of species Takydromus wol...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Animalia, kingdom of multicellular eukaryotic..."


In [ ]:
assert False

AssertionError: 

In [ ]:
strict_animalia[['genus'] + [col for col in strict_animalia.columns if col[:5]=='taxon' and len(col)<10]].iloc[0:3,:].apply(lambda x: x, axis=1)

,genus,taxon_b,taxon_c,taxon_d,taxon_e,taxon_f,taxon_g,taxon_h,taxon_i,taxon_j,...,taxon_aa,taxon_ab,taxon_ac,taxon_ad,taxon_ae,taxon_af,taxon_ag,taxon_ah,taxon_ai,taxon_aj
0,Allopeas,Subulininae,Achatinidae,Achatinoidea,Achatinina,Stylommatophora,Eupulmonata,Tectipleura,Euthyneura,Heterobranchia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Odontomantis,Hymenopodidae,Mantodea,Dictyoptera,Neoptera,Metapterygota,Pterygota,Insecta,Arthropoda,Invertebrata,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cavia,Caviinae,Caviidae,Hystricognathi,Hystricognatha,Rodentia,Glires,Gliriformes,Euarchontoglires,Boreoeutheria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
'''
samp = strict_animalia

# dropped due to token limit  + [f"Here is {x['species']}'s taxonomic tree: " + '; '.join(x['extra_captions'])]
NUM = 3
o = samp.apply(lambda x: [x['species'] + ' belongs to ' + x['extra_captions'][i] + '. It also belongs to '  + '. It also belongs to '.join(x['extra_captions'][i+1:i+NUM]).replace('..','.') +'.' for i in range(0, len(x['extra_captions']), NUM) if x['extra_captions'] is not None], axis=1) 
p = samp.apply(lambda x: f"Here is {x['species']}'s taxonomic tree: {', '.join([wo.split(', ')[0] for wo in x['extra_captions']])}."  if x['extra_captions'] is not None else None, axis=1)
p.iloc[0]
# list(map(lambda x: len(x), p))
'''

KeyboardInterrupt: 

In [ ]:
o = strict_animalia.apply(lambda x: [x['species'] + ' belongs to ' + x['extra_captions'][i] + '.' for i in range(len(x['extra_captions']))], axis=1)

In [ ]:
o[0]

['Panthera leo belongs to Ferae, clade of mammalsFeliformia, suborder of mammals in the order Carnivora. It also belongs to Carnivora, order of mammals. It also belongs to Ferae, clade of mammals.',
 'Panthera leo belongs to Felidae, family of mammals in the order of carnivoransPanthera, genus of mammals : wild cats. It also belongs to Pantherinae, subfamily of mammals. It also belongs to Felidae, family of mammals in the order of carnivorans.']

In [ ]:
o.iloc[0]

['Panthera leo belongs to Panthera, genus of mammals : wild cats. It also belongs to Pantherinae, subfamily of mammals. It also belongs to Panthera leo belongs to Felidae, family of mammals in the order of carnivorans. It also belongs to Feliformia, suborder of mammals in the order Carnivora..']

In [ ]:
o[0]

['Panthera leo belongs to Ferae, clade of mammals. It also belongs to Carnivora, order of mammals. It also belongs to Feliformia, suborder of mammals in the order Carnivora.',
 'Panthera leo belongs to Felidae, family of mammals in the order of carnivorans. It also belongs to Pantherinae, subfamily of mammals. It also belongs to Panthera, genus of mammals : wild cats.']

In [ ]:
strict_animalia['extra_captions'].iloc[0]
todrop

['genus',
 'taxon_b',
 'taxon_c',
 'taxon_d',
 'taxon_e',
 'parent_taxon_x',
 'id_x',
 'genus_parent',
 'genus_caption',
 'id_y',
 'taxon_b_parent',
 'taxon_b_caption',
 'id_x',
 'taxon_c_parent',
 'taxon_c_caption',
 'id_y',
 'taxon_d_parent',
 'taxon_d_caption',
 'id',
 'taxon_e_parent',
 'taxon_e_caption']

In [ ]:
strict_animalia['extra_captions'] = o

In [ ]:
strict_animalia['extra_captions'].map(lambda x: [w[:273] for w in x])

0         [Allopeas clavulinum belongs to Animalia, king...
1         [Odontomantis planiceps belongs to Animalia, k...
2         [Cavia porcellus belongs to Animalia, kingdom ...
3         [Tenebrio molitor belongs to Animalia, kingdom...
4         [Vespa affinis belongs to Animalia, kingdom of...
                                ...                        
137700    [Pyrocoelia rufa belongs to Animalia, kingdom ...
137701    [Acheilognathus hondae belongs to Animalia, ki...
137702    [Agelastica coerulea belongs to Animalia, king...
137703    [Takydromus wolteri belongs to Animalia, kingd...
137704    [Tajuria igolotiana belongs to Animalia, kingd...
Name: extra_captions, Length: 137593, dtype: object

In [ ]:
strict_animalia['combined_captions'] = strict_animalia.apply(lambda x: x['extra_captions'], axis=1)

In [ ]:
strict_animalia['combined_captions'] = strict_animalia['combined_captions'].map(lambda x: [w[:273] for w in x])

In [ ]:
strict_animalia['combined_captions'].iloc[2000]

['Toana caustipennis belongs to Animalia, kingdom of multicellular eukaryotic organisms.',
 'Toana caustipennis belongs to Arthropoda, phylum of invertebrates with jointed exoskeletons.',
 'Toana caustipennis belongs to Insecta, class of arthropods.',
 'Toana caustipennis belongs to Lepidoptera, order of insects that includes butterflies and moths.',
 'Toana caustipennis belongs to Noctuidae, family of insects.',
 'Toana caustipennis belongs to Toana, genus of insects.']

In [ ]:
from ast import literal_eval
strict_animalia['captions'] = strict_animalia['captions'].map(lambda x: literal_eval(x))

In [ ]:
strict_animalia['new_captions'] = strict_animalia.apply(lambda x: x['captions']+x['combined_captions'], axis=1)

In [ ]:
# wnew_4 = wnew_4.drop(todrop, axis=1)
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes,extra_captions,combined_captions,new_captions
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,[This is an example of species Allopeas clavul...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Allopeas clavulinum belongs to Animalia, king...","[Allopeas clavulinum belongs to Animalia, king...",[This is an example of species Allopeas clavul...
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,[This is an example of species Odontomantis pl...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Odontomantis planiceps belongs to Animalia, k...","[Odontomantis planiceps belongs to Animalia, k...",[This is an example of species Odontomantis pl...
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,[This is an example of species Cavia porcellus...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Cavia porcellus belongs to Animalia, kingdom ...","[Cavia porcellus belongs to Animalia, kingdom ...",[This is an example of species Cavia porcellus...
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,[This is an example of species Tenebrio molito...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Tenebrio molitor belongs to Animalia, kingdom...","[Tenebrio molitor belongs to Animalia, kingdom...",[This is an example of species Tenebrio molito...
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,"[This is an example of species Vespa affinis.,...",Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Vespa affinis belongs to Animalia, kingdom of...","[Vespa affinis belongs to Animalia, kingdom of...","[This is an example of species Vespa affinis.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,[This is an example of species Pyrocoelia rufa...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Pyrocoelia rufa belongs to Animalia, kingdom ...","[Pyrocoelia rufa belongs to Animalia, kingdom ...",[This is an example of species Pyrocoelia rufa...
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,[This is an example of species Acheilognathus ...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Acheilognathus hondae belongs to Animalia, ki...","[Acheilognathus hondae belongs to Animalia, ki...",[This is an example of species Acheilognathus ...
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,[This is an example of species Agelastica coer...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Agelastica coerulea belongs to Animalia, king...","[Agelastica coerulea belongs to Animalia, king...",[This is an example of species Agelastica coer...
137703,137801,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,[This is an example of species Takydromus wolt...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Takydromus wolteri belongs to Animalia, kingd...","[Takydromus wolteri b

In [ ]:
df = pd.concat([pd.read_csv('data/image_captions_0.csv'), pd.read_csv('data/image_captions_1.csv'), pd.read_csv('data/image_captions_2.csv'), pd.read_csv('data/image_captions_3.csv'), pd.read_csv('data/image_captions_4.csv')])
df

,filename,species,captions,parent_taxon,source,image_captions
0,Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,English: Lion (Panthera leo) lying down in Na...
1,Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,"English: Lioness at Okonjima Lodge, Namibia"
2,Squirrel_posing.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,English: A red squirrel in the forest (Sciuru...
3,Ekorre_8218.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,"Svenska: Svensk Ekorre, Vaxholm, SverigeTempl..."
4,Hepatitis_B_virus_1.jpg,Hepatitis B virus,['This is an example of species Hepatitis B vi...,Q6746239,wikimedia,English: Transmission electron micrograph of ...
...,...,...,...,...,...,...
55298,Psalidognathus_sallei_2.jpg,Psalidognathus sallei,['This is an example of species Psalidognathus...,Q1756213,wikimedia,English: Psalidognathus sallei Venezuela
55299,Hemiptarsenus_jilinus_(10.3897-zookeys.1033.62...,Hemiptarsenus jilinus,['This is an example of species Hemiptarsenus ...,Q13983711,wikimedia,English: Figure 2. \nHemiptarsenus jilinus sp...
0,Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Türkçe: Campanula adiyamanensis
1,"Cercopis_septemmaculata_(DSCN0578,_crop_0480x0...",Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,English: Cercopis septemmaculata


In [ ]:
df[df['species']=="Glyphodes bivitralis"]['image_captions']

16902    Glyphodes bivitralis(Guenée, 1854) from Mangalore
Name: image_captions, dtype: object

In [ ]:
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes,extra_captions,combined_captions,new_captions
0,0,"taxo_data_images/""+arya+""_Allopeas_clavulinum_...",Allopeas clavulinum,[This is an example of species Allopeas clavul...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Allopeas clavulinum belongs to Animalia, king...","[Allopeas clavulinum belongs to Animalia, king...",[This is an example of species Allopeas clavul...
1,1,"taxo_data_images/""+arya+""_Belalang_semut_(Odon...",Odontomantis planiceps,[This is an example of species Odontomantis pl...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Odontomantis planiceps belongs to Animalia, k...","[Odontomantis planiceps belongs to Animalia, k...",[This is an example of species Odontomantis pl...
2,2,"taxo_data_images/""+arya+""_Cavia_porcellus_-_ti...",Cavia porcellus,[This is an example of species Cavia porcellus...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Cavia porcellus belongs to Animalia, kingdom ...","[Cavia porcellus belongs to Animalia, kingdom ...",[This is an example of species Cavia porcellus...
3,3,"taxo_data_images/""+arya+""_Tenebrio_molitor_ꦲꦸꦭ...",Tenebrio molitor,[This is an example of species Tenebrio molito...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Tenebrio molitor belongs to Animalia, kingdom...","[Tenebrio molitor belongs to Animalia, kingdom...",[This is an example of species Tenebrio molito...
4,4,"taxo_data_images/""+arya+""_Vespa_affinis_2021_1...",Vespa affinis,"[This is an example of species Vespa affinis.,...",Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Vespa affinis belongs to Animalia, kingdom of...","[Vespa affinis belongs to Animalia, kingdom of...","[This is an example of species Vespa affinis.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,taxo_data_images/반딧불이.jpg,Pyrocoelia rufa,[This is an example of species Pyrocoelia rufa...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Pyrocoelia rufa belongs to Animalia, kingdom ...","[Pyrocoelia rufa belongs to Animalia, kingdom ...",[This is an example of species Pyrocoelia rufa...
137701,137799,taxo_data_images/서호납줄갱이.jpg,Acheilognathus hondae,[This is an example of species Acheilognathus ...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Acheilognathus hondae belongs to Animalia, ki...","[Acheilognathus hondae belongs to Animalia, ki...",[This is an example of species Acheilognathus ...
137702,137800,taxo_data_images/오리나무잎벌레.png,Agelastica coerulea,[This is an example of species Agelastica coer...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Agelastica coerulea belongs to Animalia, king...","[Agelastica coerulea belongs to Animalia, king...",[This is an example of species Agelastica coer...
137703,137801,taxo_data_images/줄장지뱀.jpg,Takydromus wolteri,[This is an example of species Takydromus wolt...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Takydromus wolteri belongs to Animalia, kingd...","[Takydromus wolteri b

In [ ]:
df.loc[df['image_captions'].str.find("DOCTYPE")!=-1, 'image_captions'] = ''

In [ ]:
df.loc[df['image_captions'].isna(), 'image_captions'] = ''

In [ ]:
set(strict_animalia['filename'].str[17:]) - set(df['filename'])

set()

In [ ]:
df['image_captions'] = df['image_captions'].str.strip('English:').str.strip()

In [ ]:
df['image_captions'] = df['image_captions'].str[:273]

In [ ]:
strz = 'taxo_data_images/'

In [ ]:
strict_animalia['filename'] = strict_animalia['filename'].str[17:]
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes,extra_captions,combined_captions,new_captions
0,0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an example of species Allopeas clavul...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Allopeas clavulinum belongs to Animalia, king...","[Allopeas clavulinum belongs to Animalia, king...",[This is an example of species Allopeas clavul...
1,1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an example of species Odontomantis pl...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Odontomantis planiceps belongs to Animalia, k...","[Odontomantis planiceps belongs to Animalia, k...",[This is an example of species Odontomantis pl...
2,2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an example of species Cavia porcellus...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Cavia porcellus belongs to Animalia, kingdom ...","[Cavia porcellus belongs to Animalia, kingdom ...",[This is an example of species Cavia porcellus...
3,3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an example of species Tenebrio molito...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Tenebrio molitor belongs to Animalia, kingdom...","[Tenebrio molitor belongs to Animalia, kingdom...",[This is an example of species Tenebrio molito...
4,4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,"[This is an example of species Vespa affinis.,...",Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Vespa affinis belongs to Animalia, kingdom of...","[Vespa affinis belongs to Animalia, kingdom of...","[This is an example of species Vespa affinis.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,반딧불이.jpg,Pyrocoelia rufa,[This is an example of species Pyrocoelia rufa...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Pyrocoelia rufa belongs to Animalia, kingdom ...","[Pyrocoelia rufa belongs to Animalia, kingdom ...",[This is an example of species Pyrocoelia rufa...
137701,137799,서호납줄갱이.jpg,Acheilognathus hondae,[This is an example of species Acheilognathus ...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Acheilognathus hondae belongs to Animalia, ki...","[Acheilognathus hondae belongs to Animalia, ki...",[This is an example of species Acheilognathus ...
137702,137800,오리나무잎벌레.png,Agelastica coerulea,[This is an example of species Agelastica coer...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Agelastica coerulea belongs to Animalia, king...","[Agelastica coerulea belongs to Animalia, king...",[This is an example of species Agelastica coer...
137703,137801,줄장지뱀.jpg,Takydromus wolteri,[This is an example of species Takydromus wolt...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Takydromus wolteri belongs to Animalia, kingd...","[Takydromus wolteri belongs to Animalia, kingd...",[This is an example of species Takydromus wolt...


In [ ]:
strict_animalia = strict_animalia.drop_duplicates(['filename', 'species'])
strict_animalia

,index,filename,species,captions,parent_taxon_x,source,id_x,genus,genus_parent,genus_caption,...,taxon_ai_parent,taxon_ai_caption,id_y,taxon_aj,taxon_aj_parent,taxon_aj_caption,match_classes,extra_captions,combined_captions,new_captions
0,0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an example of species Allopeas clavul...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Allopeas clavulinum belongs to Animalia, king...","[Allopeas clavulinum belongs to Animalia, king...",[This is an example of species Allopeas clavul...
1,1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an example of species Odontomantis pl...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Odontomantis planiceps belongs to Animalia, k...","[Odontomantis planiceps belongs to Animalia, k...",[This is an example of species Odontomantis pl...
2,2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an example of species Cavia porcellus...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Cavia porcellus belongs to Animalia, kingdom ...","[Cavia porcellus belongs to Animalia, kingdom ...",[This is an example of species Cavia porcellus...
3,3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an example of species Tenebrio molito...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Tenebrio molitor belongs to Animalia, kingdom...","[Tenebrio molitor belongs to Animalia, kingdom...",[This is an example of species Tenebrio molito...
4,4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,"[This is an example of species Vespa affinis.,...",Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Vespa affinis belongs to Animalia, kingdom of...","[Vespa affinis belongs to Animalia, kingdom of...","[This is an example of species Vespa affinis.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137700,137798,반딧불이.jpg,Pyrocoelia rufa,[This is an example of species Pyrocoelia rufa...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Pyrocoelia rufa belongs to Animalia, kingdom ...","[Pyrocoelia rufa belongs to Animalia, kingdom ...",[This is an example of species Pyrocoelia rufa...
137701,137799,서호납줄갱이.jpg,Acheilognathus hondae,[This is an example of species Acheilognathus ...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Acheilognathus hondae belongs to Animalia, ki...","[Acheilognathus hondae belongs to Animalia, ki...",[This is an example of species Acheilognathus ...
137702,137800,오리나무잎벌레.png,Agelastica coerulea,[This is an example of species Agelastica coer...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Agelastica coerulea belongs to Animalia, king...","[Agelastica coerulea belongs to Animalia, king...",[This is an example of species Agelastica coer...
137703,137801,줄장지뱀.jpg,Takydromus wolteri,[This is an example of species Takydromus wolt...,Q1805394,wikimedia,Q1805394,Takydromus,Q649583,genus of reptiles,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Takydromus wolteri belongs to Animalia, kingd...","[Takydromus wolteri belongs to Animalia, kingd...",[This is an example of species Takydromus wolt...


In [ ]:
df = df.drop_duplicates(['filename', 'species'])
df

,filename,species,captions,parent_taxon,source,image_captions
0,Lion_waiting_in_Namibia.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,Lion (Panthera leo) lying down in Namibia. Pho...
1,Okonjima_Lioness.jpg,Panthera leo,['This is an example of species Panthera leo.'...,Q127960,wikimedia,"Lioness at Okonjima Lodge, Namibia"
2,Squirrel_posing.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,A red squirrel in the forest (Sciurus vulgaris).
3,Ekorre_8218.jpg,Sciurus vulgaris,['This is an example of species Sciurus vulgar...,Q281124,wikimedia,"Svenska: Svensk Ekorre, Vaxholm, SverigeTempl..."
4,Hepatitis_B_virus_1.jpg,Hepatitis B virus,['This is an example of species Hepatitis B vi...,Q6746239,wikimedia,Transmission electron micrograph of Hepatitis ...
...,...,...,...,...,...,...
55298,Psalidognathus_sallei_2.jpg,Psalidognathus sallei,['This is an example of species Psalidognathus...,Q1756213,wikimedia,Psalidognathus sallei Venezuela
55299,Hemiptarsenus_jilinus_(10.3897-zookeys.1033.62...,Hemiptarsenus jilinus,['This is an example of species Hemiptarsenus ...,Q13983711,wikimedia,"Figure 2. \nHemiptarsenus jilinus sp. nov., ma..."
0,Campanula_adiyamanensis.jpg,Campanula adiyamanensis,['This is an example of species Campanula adiy...,Q157069,wikimedia,Türkçe: Campanula adiyamane
1,"Cercopis_septemmaculata_(DSCN0578,_crop_0480x0...",Cercopis septemmaculata,['This is an example of species Cercopis septe...,Q2946582,wikimedia,Cercopis septemmaculata


In [ ]:
strict_animalia = strict_animalia.merge(df, how='left', on=['species', 'filename'])
strict_animalia

,index,filename,species,captions_x,parent_taxon_x,source_x,id_x,genus,genus_parent,genus_caption,...,taxon_aj_parent,taxon_aj_caption,match_classes,extra_captions,combined_captions,new_captions,captions_y,parent_taxon,source_y,image_captions
0,0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an example of species Allopeas clavul...,Q3738109,wikimedia,Q3738109,Allopeas,Q47123120,genus of molluscs,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Allopeas clavulinum belongs to Animalia, king...","[Allopeas clavulinum belongs to Animalia, king...",[This is an example of species Allopeas clavul...,['This is an example of species Allopeas clavu...,Q3738109,wikimedia,Bahasa Indonesia: Allopeas clavulinum
1,1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an example of species Odontomantis pl...,Q10607011,wikimedia,Q10607011,Odontomantis,Q592684,genus of mantises,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Odontomantis planiceps belongs to Animalia, k...","[Odontomantis planiceps belongs to Animalia, k...",[This is an example of species Odontomantis pl...,['This is an example of species Odontomantis p...,Q10607011,wikimedia,Bahasa Indonesia: Belalang semut (Odontomanti...
2,2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an example of species Cavia porcellus...,Q286088,wikimedia,Q286088,Cavia,Q914293,genus of mammals,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Cavia porcellus belongs to Animalia, kingdom ...","[Cavia porcellus belongs to Animalia, kingdom ...",[This is an example of species Cavia porcellus...,['This is an example of species Cavia porcellu...,Q286088,wikimedia,Bahasa Indonesia: Cavia porcellus janta
3,3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an example of species Tenebrio molito...,Q3518056,wikimedia,Q3518056,Tenebrio,Q615799,genus of insects,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Tenebrio molitor belongs to Animalia, kingdom...","[Tenebrio molitor belongs to Animalia, kingdom...",[This is an example of species Tenebrio molito...,['This is an example of species Tenebrio molit...,Q3518056,wikimedia,Tenebrio molitor
4,4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,"[This is an example of species Vespa affinis.,...",Q334422,wikimedia,Q334422,Vespa,Q1076176,genus of insects,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Vespa affinis belongs to Animalia, kingdom of...","[Vespa affinis belongs to Animalia, kingdom of...","[This is an example of species Vespa affinis.,...",['This is an example of species Vespa affinis....,Q334422,wikimedia,Bahasa Indonesia: Vespa aff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137588,137798,반딧불이.jpg,Pyrocoelia rufa,[This is an example of species Pyrocoelia rufa...,Q7733181,wikimedia,Q7733181,Pyrocoelia,Q25420,genus of insects,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Pyrocoelia rufa belongs to Animalia, kingdom ...","[Pyrocoelia rufa belongs to Animalia, kingdom ...",[This is an example of species Pyrocoelia rufa...,['This is an example of species Pyrocoelia ruf...,Q7733181,wikimedia,한국어: 반딧불이
137589,137799,서호납줄갱이.jpg,Acheilognathus hondae,[This is an example of species Acheilognathus ...,Q481845,wikimedia,Q481845,Acheilognathus,Q35047,genus of fishes,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell...","[Acheilognathus hondae belongs to Animalia, ki...","[Acheilognathus hondae belongs to Animalia, ki...",[This is an example of species Acheilognathus ...,['This is an example of species Acheilognathus...,Q481845,wikimedia,한국어: 서호납줄갱이 사진
137590,137800,오리나무잎벌레.png,Agelastica coerulea,[This is an example of species Agelastica coer...,Q1941116,wikimedia,Q1941116,Agelastica,Q2842229,genus of leaf beetles,...,NaN,NaN,"{'kingdom': ('Animalia', 'kingdom of multicell.

In [ ]:
strict_animalia['captions_x'].iloc[0:1].map(lambda x: "This is an " + x[1][0].lower() + x[1][1:] + " This is its species name, " + x[2][0].lower() + ' - '.join(x[2][1:].split('. '))).iloc[0]

'This is an image of an animal with taxon name: Allopeas clavulinum. This is its species name, species: Allopeas clavulinum - species of molluscs'

In [ ]:
strict_animalia['final_captions'].iloc[100]

['This is an image of an animal with taxon name: Colias myrmidone. This is its species name, species: Colias myrmidone - species of butterfly.',
 'Colias myrmidone belongs to Animalia, kingdom of multicellular eukaryotic organisms.',
 'Colias myrmidone belongs to Arthropoda, phylum of invertebrates with jointed exoskeletons.',
 'Colias myrmidone belongs to Insecta, class of arthropods.',
 'Colias myrmidone belongs to Lepidoptera, order of insects that includes butterflies and moths.',
 'Colias myrmidone belongs to Pieridae, family of insects.',
 'Colias myrmidone belongs to Colias, genus of insects.']

In [ ]:
strict_animalia['final_captions'] = strict_animalia['new_captions'].map(lambda x: ["This is an " + x[1][0].lower() + x[1][1:] + " This is its species name, " + x[2][0].lower() + ' - '.join(x[2][1:].split('. ')) + '.'] + x[5:])

In [ ]:
strict_animalia['final_captions'].map(lambda x: len(x)).value_counts()

7    117862
6     18803
5       873
4        49
3         6
Name: final_captions, dtype: int64

In [ ]:
strict_animalia = strict_animalia[strict_animalia.final_captions.map(len)>4]

In [ ]:
strict_animalia_out = strict_animalia[['filename', 'species', 'final_captions', 'image_captions']]
strict_animalia_out

,filename,species,final_captions,image_captions
0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an image of an animal with taxon name...,Bahasa Indonesia: Allopeas clavulinum
1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an image of an animal with taxon name...,Bahasa Indonesia: Belalang semut (Odontomanti...
2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an image of an animal with taxon name...,Bahasa Indonesia: Cavia porcellus janta
3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an image of an animal with taxon name...,Tenebrio molitor
4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,[This is an image of an animal with taxon name...,Bahasa Indonesia: Vespa aff
...,...,...,...,...
137588,반딧불이.jpg,Pyrocoelia rufa,[This is an image of an animal with taxon name...,한국어: 반딧불이
137589,서호납줄갱이.jpg,Acheilognathus hondae,[This is an image of an animal with taxon name...,한국어: 서호납줄갱이 사진
137590,오리나무잎벌레.png,Agelastica coerulea,[This is an image of an animal with taxon name...,한국어: 오리나무잎벌레의 사진
137591,줄장지뱀.jpg,Takydromus wolteri,[This is an image of an animal with taxon name...,조선말: 배추 위의 줄장지뱀


In [ ]:
strict_animalia_out['final_captions'] = strict_animalia_out.apply(lambda x: x['final_captions']+[x['image_captions']], axis=1)
strict_animalia_out

C:\Users\james\AppData\Local\Temp\ipykernel_12888\2912134766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strict_animalia_out['final_captions'] = strict_animalia_out.apply(lambda x: x['final_captions']+[x['image_captions']], axis=1)


,filename,species,final_captions,image_captions
0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an image of an animal with taxon name...,Bahasa Indonesia: Allopeas clavulinum
1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an image of an animal with taxon name...,Bahasa Indonesia: Belalang semut (Odontomanti...
2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an image of an animal with taxon name...,Bahasa Indonesia: Cavia porcellus janta
3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an image of an animal with taxon name...,Tenebrio molitor
4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,[This is an image of an animal with taxon name...,Bahasa Indonesia: Vespa aff
...,...,...,...,...
137588,반딧불이.jpg,Pyrocoelia rufa,[This is an image of an animal with taxon name...,한국어: 반딧불이
137589,서호납줄갱이.jpg,Acheilognathus hondae,[This is an image of an animal with taxon name...,한국어: 서호납줄갱이 사진
137590,오리나무잎벌레.png,Agelastica coerulea,[This is an image of an animal with taxon name...,한국어: 오리나무잎벌레의 사진
137591,줄장지뱀.jpg,Takydromus wolteri,[This is an image of an animal with taxon name...,조선말: 배추 위의 줄장지뱀


In [ ]:
strict_animalia_out.final_captions.map(lambda x: len(x)).value_counts()

8    117862
7     18803
6       873
Name: final_captions, dtype: int64

In [ ]:
eval_df = strict_animalia[['filename', 'species', 'genus', 'genus_caption']+[col for col in strict_animalia.columns if col.find('taxon_')==0 and col.find('parent')==-1]]
eval_df['filename'] = eval_df['filename'].map(lambda x: strz+x)

C:\Users\james\AppData\Local\Temp\ipykernel_12888\81307260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df['filename'] = eval_df['filename'].map(lambda x: strz+x)


In [ ]:
eval_df.to_csv('eval_df_test_resource.csv', index=False, encoding='utf-8')

In [ ]:
df['image_captions'].map(lambda x: type(x)).value_counts()

<class 'str'>    221188
Name: image_captions, dtype: int64

In [ ]:
strict_animalia_out[['filename', 'species', 'final_captions']]

,filename,species,final_captions
0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an image of an animal with taxon name...
1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an image of an animal with taxon name...
2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an image of an animal with taxon name...
3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an image of an animal with taxon name...
4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,[This is an image of an animal with taxon name...
...,...,...,...
137588,반딧불이.jpg,Pyrocoelia rufa,[This is an image of an animal with taxon name...
137589,서호납줄갱이.jpg,Acheilognathus hondae,[This is an image of an animal with taxon name...
137590,오리나무잎벌레.png,Agelastica coerulea,[This is an image of an animal with taxon name...
137591,줄장지뱀.jpg,Takydromus wolteri,[This is an image of an animal with taxon name...


In [ ]:
assert False

In [ ]:
strict_animalia_out['filename'] = strict_animalia_out['filename'].map(lambda x: strz+x)

C:\Users\james\AppData\Local\Temp\ipykernel_12888\1119196489.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strict_animalia_out['filename'] = strict_animalia_out['filename'].map(lambda x: strz+x)


In [ ]:
strict_animalia_out[['filename', 'species', 'final_captions']].rename({'final_captions': 'captions'}, axis=1)

,filename,species,captions
0,"""+arya+""_Allopeas_clavulinum_-_siput_duri_-_Te...",Allopeas clavulinum,[This is an image of an animal with taxon name...
1,"""+arya+""_Belalang_semut_(Odontomantis_javana)_...",Odontomantis planiceps,[This is an image of an animal with taxon name...
2,"""+arya+""_Cavia_porcellus_-_tikus_belanda_01.jpg",Cavia porcellus,[This is an image of an animal with taxon name...
3,"""+arya+""_Tenebrio_molitor_ꦲꦸꦭꦼꦂ_ꦏꦤ꧀ꦢꦁ_ulat_kan...",Tenebrio molitor,[This is an image of an animal with taxon name...
4,"""+arya+""_Vespa_affinis_2021_1.jpg",Vespa affinis,[This is an image of an animal with taxon name...
...,...,...,...
137588,반딧불이.jpg,Pyrocoelia rufa,[This is an image of an animal with taxon name...
137589,서호납줄갱이.jpg,Acheilognathus hondae,[This is an image of an animal with taxon name...
137590,오리나무잎벌레.png,Agelastica coerulea,[This is an image of an animal with taxon name...
137591,줄장지뱀.jpg,Takydromus wolteri,[This is an image of an animal with taxon name...


In [ ]:
strict_animalia_out[['filename', 'species', 'final_captions']].rename({'final_captions': 'new_captions'}, axis=1).to_csv('full_captions_taxons_dataset_final_final.csv', index=False, encoding='utf-8')

In [ ]:
out = strict_animalia_out[['filename', 'species', 'final_captions']].rename({'final_captions': 'captions'}, axis=1).to_dict(orient='records')
# .to_json('full_captions_taxons_dataset.json', orient='records')

In [ ]:
import json

{'filename': 'Lion_waiting_in_Namibia.jpg',
 'species': 'Panthera leo',
 'new_captions': ['This is an image of an animal with taxon name: Panthera leo. This is its species name, species: Panthera leo - species of big cat.',
  'Panthera leo belongs to Ferae, clade of mammals. It also belongs to Carnivora, order of mammals. It also belongs to Feliformia, suborder of mammals in the order Carnivora.',
  'Panthera leo belongs to Felidae, family of mammals in the order of carnivorans. It also belongs to Pantherinae, subfamily of mammals. It also belongs to Panthera, genus of mammals : wild cats.',
  "Here is Panthera leo's taxonomic tree: Ferae, Carnivora, Feliformia, Felidae, Pantherinae, Panthera.",
  'Lion (Panthera leo) lying down in Namibia. Photographer\'s comment: "[I was] standing on a viewing platform about 25m away. He wasn\'t doing much at all other than looking regal. There where [sic] some lionesses down below growling and pacing but they were in the shade so I ']}

In [ ]:
out[12019]

{'filename': 'taxo_data_images/Aquilonastra_richmondi.jpg',
 'species': 'Aquilonastra richmondi',
 'captions': ['This is an image of an animal with taxon name: Aquilonastra richmondi. This is its species name, species: Aquilonastra richmondi - species of echinoderm.',
  'Aquilonastra richmondi belongs to Animalia, kingdom of multicellular eukaryotic organisms.',
  'Aquilonastra richmondi belongs to Echinodermata, phylum of animals.',
  'Aquilonastra richmondi belongs to Asteroidea, class of echinoderms, marine animal.',
  'Aquilonastra richmondi belongs to Spinulosida, order of echinoderms.',
  'Aquilonastra richmondi belongs to Asterinidae, family of echinoderms.',
  'Aquilonastra richmondi belongs to Aquilonastra, genus of echinoderms.',
  'Français\xa0:  Aquilonastra richmondi à la Réunion.']}

In [ ]:
with open("full_captions_taxons_dataset_final_final.json", 'w', encoding='utf-8') as f:
    json.dump(out, f, ensure_ascii = True, allow_nan=False)

In [ ]:
wnew_4_out[['filename', 'species', 'new_captions']].to_json('full_captions_taxons_dataset.ndjson', orient='records', lines=True)